In [22]:
import ee
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

In [24]:
# If it's for the first time, authentication is required before running this code
# https://developers.google.com/earth-engine/guides/python_install-conda#windows_5
ee.Initialize()

In [25]:
# earth engine location of the subsets of the 10km grid created separately in the script "00_create_10_by_10_grid.R"
# 10km grid shapefile needs to be uploaded to assets in your GEE before running this line
grid = ee.FeatureCollection("projects/ee-akawano/assets/grid_10km")

# google drive associated with your GEE account
output_drive = "s5p_NO2_L3_10km"
timezone = "utc"

In [26]:
# Sentinel-5P OFFL NO2: Offline Nitrogen Dioxide
# https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S5P_OFFL_L3_NO2
# 2018-06-28 to 2022-12-12
maiac = ee.ImageCollection("COPERNICUS/S5P/OFFL/L3_NO2").select("NO2_column_number_density")
maiac_proj = maiac.first().projection()
pixel_res = maiac_proj.nominalScale().getInfo()

In [27]:
# set start_date and end_date
start_date_1 = pd.to_datetime("2018-06-28", format="%Y/%m/%d")
end_date_1 = pd.to_datetime("2022-08-01", format="%Y/%m/%d")

# Initialize an empty list to store the date strings for the first list
start_date_list = []

# Loop through the dates for the first list, incrementing by 3 months each time
while start_date_1 < end_date_1:
    start_date_list.append(start_date_1.strftime("%Y-%m-%d"))
    
    # Increment the start date by 3 months
    start_date_1 += relativedelta(months=3)

# Start and end dates for the second list
start_date_2 = pd.to_datetime("2018-10-01", format="%Y/%m/%d")
end_date_2 = pd.to_datetime("2022-11-01", format="%Y/%m/%d")

# Initialize an empty list to store the date strings for the second list
end_date_list = []

# Loop through the dates for the second list, incrementing by 3 months each time
while start_date_2 < end_date_2:
    # Format the date as a string in the year-month-day format
    end_date_list.append(start_date_2.strftime("%Y-%m-%d"))
    
    # Increment the start date by 3 months
    start_date_2 += relativedelta(months=3)

# Print the lists of date strings
print(start_date_list)
print(end_date_list)

['2018-06-28', '2018-09-28', '2018-12-28', '2019-03-28', '2019-06-28', '2019-09-28', '2019-12-28', '2020-03-28', '2020-06-28', '2020-09-28', '2020-12-28', '2021-03-28', '2021-06-28', '2021-09-28', '2021-12-28', '2022-03-28', '2022-06-28']
['2018-10-01', '2019-01-01', '2019-04-01', '2019-07-01', '2019-10-01', '2020-01-01', '2020-04-01', '2020-07-01', '2020-10-01', '2021-01-01', '2021-04-01', '2021-07-01', '2021-10-01', '2022-01-01', '2022-04-01', '2022-07-01', '2022-10-01']


In [28]:
# Calculate how many days between start and end days
def calculate_length(start_date, end_date):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    length_date = (end_date - start_date).days
    return length_date

In [29]:
def create_image(start_date, length_date):
    images_list = ee.List([])
    for i in range(0, length_date):
        # timezone is default utc in ee.Date.fromYMD
        start_date = pd.to_datetime(start_date)
        im_date = ee.Date.fromYMD(start_date.year, start_date.month, start_date.day).advance(i, 'day')
        im_date_end = im_date.advance(1, 'day')
        im = maiac.filterDate(im_date, im_date_end).reduce(ee.Reducer.mean())
        im = ee.Image(im).set("start_date", im_date.format("yMMdd"))
        images_list = images_list.add(im)
    return ee.ImageCollection.fromImages(images_list)

In [30]:
# For each day, calculate average for each cell grid
def process_image(start_date, end_date):
    length_date = calculate_length(start_date, end_date)
    maiac_images = create_image(start_date, length_date)
    maiac_images_reduced = maiac_images.map(lambda daily_im: (
        daily_im.reduceRegions(
            collection = grid,
            reducer = ee.Reducer.mean(),
            crs = maiac_proj,
            scale = pixel_res
        ).map(lambda f: f.set("start_date", daily_im.get("start_date")))))
    grid_maiac = ee.FeatureCollection(maiac_images_reduced).flatten()
    return grid_maiac

In [31]:
# make a list of properties to export 
export_properties = ["grid_id", "mean", "start_date"]
export_properties

['grid_id', 'mean', 'start_date']

In [10]:
# loop using start_date_list and end_date_list
for start_date, end_date in zip(start_date_list, end_date_list):
        grid_NO2 = process_image(start_date, end_date)
        x = "NO2_10km_grid"
        y = start_date
        t = "to"
        z = end_date
        output_name = "_".join([x, y, t, z])
        aod_task = ee.batch.Export.table.toDrive(
            collection = grid_NO2,
            folder = output_drive,
            description = output_name,
            fileFormat = "CSV",
            selectors = export_properties)
        print(f"saving file as", {output_name})
        aod_task.start()

saving file as {'NO2_10km_grid_2018-06-28_to_2018-10-01'}
saving file as {'NO2_10km_grid_2018-09-28_to_2019-01-01'}
saving file as {'NO2_10km_grid_2018-12-28_to_2019-04-01'}
saving file as {'NO2_10km_grid_2019-03-28_to_2019-07-01'}
saving file as {'NO2_10km_grid_2019-06-28_to_2019-10-01'}
saving file as {'NO2_10km_grid_2019-09-28_to_2020-01-01'}
saving file as {'NO2_10km_grid_2019-12-28_to_2020-04-01'}
saving file as {'NO2_10km_grid_2020-03-28_to_2020-07-01'}
saving file as {'NO2_10km_grid_2020-06-28_to_2020-10-01'}
saving file as {'NO2_10km_grid_2020-09-28_to_2021-01-01'}
saving file as {'NO2_10km_grid_2020-12-28_to_2021-04-01'}
saving file as {'NO2_10km_grid_2021-03-28_to_2021-07-01'}
saving file as {'NO2_10km_grid_2021-06-28_to_2021-10-01'}
saving file as {'NO2_10km_grid_2021-09-28_to_2022-01-01'}
saving file as {'NO2_10km_grid_2021-12-28_to_2022-04-01'}
saving file as {'NO2_10km_grid_2022-03-28_to_2022-07-01'}
saving file as {'NO2_10km_grid_2022-06-28_to_2022-10-01'}


In [17]:
start_date = '2022-10-01'
end_date = '2022-11-01'

grid_NO2 = process_image(start_date, end_date)
x = "NO2_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_NO2,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'NO2_10km_grid_2022-10-01_to_2022-11-01'}


In [32]:
# Error: NO2_10km_grid_2022-03-28_to_2022-07-01
start_date = '2022-03-28'
end_date = '2022-05-01'

grid_NO2 = process_image(start_date, end_date)
x = "NO2_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_NO2,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'NO2_10km_grid_2022-03-28_to_2022-05-01'}


In [33]:
# Error: NO2_10km_grid_2022-03-28_to_2022-07-01
start_date = '2022-05-01'
end_date = '2022-07-01'

grid_NO2 = process_image(start_date, end_date)
x = "NO2_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_NO2,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'NO2_10km_grid_2022-05-01_to_2022-07-01'}


In [34]:
# Error NO2_10km_grid_2022-05-01_to_2022-07-01
start_date = '2022-05-01'
end_date = '2022-06-01'

grid_NO2 = process_image(start_date, end_date)
x = "NO2_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_NO2,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'NO2_10km_grid_2022-05-01_to_2022-06-01'}


In [35]:
# Error NO2_10km_grid_2022-05-01_to_2022-07-01
start_date = '2022-06-01'
end_date = '2022-07-01'

grid_NO2 = process_image(start_date, end_date)
x = "NO2_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_NO2,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'NO2_10km_grid_2022-06-01_to_2022-07-01'}


In [36]:
# Error NO2_10km_grid_2022-05-01_to_2022-06-01
start_date = '2022-05-01'
end_date = '2022-05-15'

grid_NO2 = process_image(start_date, end_date)
x = "NO2_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_NO2,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'NO2_10km_grid_2022-05-01_to_2022-05-15'}


In [37]:
# Error NO2_10km_grid_2022-05-01_to_2022-06-01
start_date = '2022-05-15'
end_date = '2022-06-01'

grid_NO2 = process_image(start_date, end_date)
x = "NO2_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_NO2,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'NO2_10km_grid_2022-05-15_to_2022-06-01'}


In [38]:
# Error NO2_10km_grid_2022-05-01_to_2022-05-15
start_date = '2022-05-01'
end_date = '2022-05-08'

grid_NO2 = process_image(start_date, end_date)
x = "NO2_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_NO2,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'NO2_10km_grid_2022-05-01_to_2022-05-08'}


In [39]:
# Error NO2_10km_grid_2022-05-01_to_2022-05-15
start_date = '2022-05-08'
end_date = '2022-05-15'

grid_NO2 = process_image(start_date, end_date)
x = "NO2_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_NO2,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'NO2_10km_grid_2022-05-08_to_2022-05-15'}


In [41]:
# Error NO2_10km_grid_2022-05-01_to_2022-05-08
start_date = '2022-05-01'
end_date = '2022-05-04'

grid_NO2 = process_image(start_date, end_date)
x = "NO2_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_NO2,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'NO2_10km_grid_2022-05-01_to_2022-05-04'}


In [42]:
# Error NO2_10km_grid_2022-05-01_to_2022-05-08
start_date = '2022-05-04'
end_date = '2022-05-08'

grid_NO2 = process_image(start_date, end_date)
x = "NO2_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_NO2,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'NO2_10km_grid_2022-05-04_to_2022-05-08'}


In [43]:
# Error NO2_10km_grid_2022-05-01_to_2022-05-04
start_date = '2022-05-01'
end_date = '2022-05-03'

grid_NO2 = process_image(start_date, end_date)
x = "NO2_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_NO2,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'NO2_10km_grid_2022-05-01_to_2022-05-03'}


In [44]:
# Error NO2_10km_grid_2022-05-01_to_2022-05-04
start_date = '2022-05-02'
end_date = '2022-05-04'

grid_NO2 = process_image(start_date, end_date)
x = "NO2_10km_grid"
y = start_date
t = "to"
z = end_date
output_name = "_".join([x, y, t, z])
aod_task = ee.batch.Export.table.toDrive(
    collection = grid_NO2,
    folder = output_drive,
    description = output_name,
    fileFormat = "CSV",
    selectors = export_properties)
print(f"saving file as", {output_name})
aod_task.start()

saving file as {'NO2_10km_grid_2022-05-02_to_2022-05-04'}


In [45]:
# Error NO2_10km_grid_2022-05-01_to_2022-05-03

In [ ]:
# if you want to cancel tasks:
# https://code.earthengine.google.com/tasks